In [190]:
import pandas as pd
import numpy as np

# Cleaning players dataframe

In [191]:
players_df = pd.read_json("tfmkt_scraper/crawled_data/players_with_bios.json")
players_df[:5]

,name,team,age,position,country,number,value,href,bio
0,Vaná Alves,Aris Limassol,33,Goalkeeper,Brazil,1,€400k,/vana-alves/profil/spieler/212198,No wiki bio found.
1,Mislav Zadro,Aris Limassol,21,Goalkeeper,Croatia,91,€250k,/mislav-zadro/profil/spieler/580418,No wiki bio found.
2,Ellinas Sofroniou,Aris Limassol,29,Goalkeeper,Cyprus,90,€100k,/ellinas-sofroniou/profil/spieler/292711,No wiki bio found.
3,Anastasios Pisiias,Aris Limassol,17,Goalkeeper,Cyprus,26,None,/anorthosis-famagusta/startseite/verein/3899/s...,No wiki bio found.
4,Connor Goldson,Aris Limassol,31,Centre-Back,England,18,€4.50m,/rangers-fc/startseite/verein/124/saison_id/2024,Connor Lambert Goldson(born 18 December 1992) ...


## Cleaning the bios

In [192]:
players_df[players_df['bio'].duplicated()]

,name,team,age,position,country,number,value,href,bio
1,Mislav Zadro,Aris Limassol,21,Goalkeeper,Croatia,91,€250k,/mislav-zadro/profil/spieler/580418,No wiki bio found.
2,Ellinas Sofroniou,Aris Limassol,29,Goalkeeper,Cyprus,90,€100k,/ellinas-sofroniou/profil/spieler/292711,No wiki bio found.
3,Anastasios Pisiias,Aris Limassol,17,Goalkeeper,Cyprus,26,None,/anorthosis-famagusta/startseite/verein/3899/s...,No wiki bio found.
5,Milosz Matysik,Aris Limassol,20,Centre-Back,Poland,17,€1.20m,/milosz-matysik/profil/spieler/751104,No wiki bio found.
7,Aboubacar Loucoubar,Aris Limassol,19,Centre-Back,Senegal,15,None,/aboubacar-loucoubar/profil/spieler/1221176,No wiki bio found.
...,...,...,...,...,...,...,...,...,...
18794,Antoni Burkiewicz,Raków Częstochowa,16,Defensive Midfield,Poland,47,None,/rakow-czestochowa-ii/startseite/verein/41149/...,No wiki bio found.
18798,Jean Carlos Silva,Raków Częstochowa,28,Left Midfield,Spain,20,€1.00m,/jean-carlos-silva/profil/spieler/282434,No wiki bio found.
18801,Ivi López,Raków Częstochowa,30,Attacking Midfield,Spain,10,€1.00m,/ivi-lopez/profil/spieler/282371,No wiki bio found.
18804,Tobiasz Mras,Raków Częstochowa,17,Attacking Midfield,Poland,77,€150k,/tobiasz-mras/profil/spieler/1143675,No wiki bio found.


In [193]:
bios_counts = players_df['bio'].value_counts()
weird_bios_idxs = bios_counts[bios_counts > 1].index

weird_bios = players_df[players_df['bio'].isin(weird_bios_idxs)]
weird_bios['bio'].value_counts()


bio
No wiki bio found.                                                                                                                                                                                                                                                                                 6674
This is an accepted version of this page                                                                                                                                                                                                                                                              8
Error fetching bio: HTTPSConnectionPool(host='en.wikipedia.org', port=443): Read timed out. (read timeout=2)                                                                                                                                                                                          8
Juninho, Portuguese for "little Junior", is a common nickname in Brazil for those whose full name includesJú

In [194]:
bios_erros = ["No wiki bio found.", 
              "This is an accepted version of this page", 
              "Error fetching bio: HTTPSConnectionPool(host='en.wikipedia.org', port=443): Read timed out. (read timeout=2)",
              "This is an accepted version of this page"]
# Remove the bios with the above errors encountered a few times
players_df['bio'].mask(players_df['bio'].isin(['No wiki bio found.']), None, inplace=True)

# Encountered a lot of time too
players_df['bio'] = players_df['bio'].apply(lambda x: None if 'may refer to' in str(x) else x)

/tmp/ipykernel_3434/440450349.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  players_df['bio'].mask(players_df['bio'].isin(['No wiki bio found.']), None, inplace=True)


In [195]:
players_df[players_df[['name','bio']].duplicated() & players_df['bio'].notna()].sort_values(by="name")[:10]

,name,team,age,position,country,number,value,href,bio
14293,Abdoulaye Kanté,Austria Vienna,19,Centre-Forward,Cote d'Ivoire,47,€200k,/racing-club-abidjan/startseite/verein/85059/s...,Abdoulaye Kanté(born 4 March 2000) is a French...
11583,Albin Krasniqi,FC Winterthur,21,Right Winger,Switzerland,17,€800k,/fc-st-gallen-1879-u21/startseite/verein/9178/...,Albin Krasniqi(born 6 April 2003) is a Swiss p...
16130,Aldair,Levski Sofia,25,Right-Back,Portugal,21,€700k,/sd-ponferradina/startseite/verein/4032/saison...,Aldair Nascimento dos Santos(Brazilian Portugu...
15916,Alexandru Stoian,FCV Farul Constanta,16,Centre-Forward,Romania,90,€100k,/fcv-farul-u18/startseite/verein/54047/saison_...,Alexandru Stoian(born 15 December 2007) is aRo...
341,Andreas Christofi,Enosis Neon Paralimniou,26,Centre-Back,Cyprus,3,€125k,/andreas-christofi/profil/spieler/936643,Andros Christofi(born 21 April 1969) is a reti...
2973,André,FC Samtredia,21,Central Midfield,Brazil,39,€50k,/se-itapirense-u20/startseite/verein/96911/sai...,André— sometimes transliterated asAndre— is th...
5618,André,Wolverhampton Wanderers,23,Defensive Midfield,Brazil,7,€25.00m,/fluminense-football-club/startseite/verein/24...,André— sometimes transliterated asAndre— is th...
10380,André Gomes,LOSC Lille,31,Central Midfield,Portugal,26,€10.00m,/without-club/startseite/verein/515/saison_id/...,André Filipe Tavares GomesComM(Portuguese pron...
7400,Andy Mitchell,Dungannon Swifts FC,30,Centre-Forward,Northern Ireland,26,€75k,/andy-mitchell/profil/spieler/213955,Andrew Mitchellis a British Conservative polit...
18249,Arthur,Bayer 04 Leverkusen,21,Right-Back,Brazil,13,€5.00m,/arthur/profil/spieler/977464,Arthuris a masculine given name of uncertain e...


In [196]:
# Example of problem : 

players_df.where(players_df['name'] == 'Igor Ivanovic').dropna()

,name,team,age,position,country,number,value,href,bio
17707,Igor Ivanovic,Tobol Kostanay,27,Right Winger,Serbia,10,€500k,/igor-ivanovic/profil/spieler/327611,Igor Ivanović(Serbian Cyrillic: Игор Ивановић;...
18608,Igor Ivanovic,Buducnost Podgorica,34,Right Winger,Montenegro,7,€275k,/igor-ivanovic/profil/spieler/192417,Igor Ivanović(Serbian Cyrillic: Игор Ивановић;...


In [197]:
# Remove the bios of the players for which we have exactly twice the same bio
duplicated_bios_mask = players_df['bio'].duplicated(keep=False) & players_df['bio'].notna()
players_df.loc[duplicated_bios_mask, 'bio'] = None

In [198]:
players_df[players_df['bio'].duplicated() & players_df['bio'].notna()].sort_values(by="name")[:10]

,name,team,age,position,country,number,value,href,bio


## Convert values (string) into numbers

In [ ]:
def convert_value_to_int(value_string) :
    if value_string is None : 
        return None
    else :
        value = value_string[1:]
        if value.endswith("m") :
            value_int = float(value[:-1]) * 1e6
        elif value.endswith("k") :
            value_int = float(value[:-1]) * 1e3
        elif value.endswith("bn") :
            value_int = float(value[:-1]) * 1e9
        
    return int(value_int)

In [200]:
for i, value in enumerate(players_df['value']) :
    value_int = convert_value_to_int(value)
    assert type(value_int) == int or value_int == None, print(value_int)

    # Show what's the output
    if i < 8 :
        print(f"original : {value}, final : {value_int}")

original : €400k, final : 400000
original : €250k, final : 250000
original : €100k, final : 100000
original : None, final : None
original : €4.50m, final : 4500000
original : €1.20m, final : 1200000
original : €500k, final : 500000
original : None, final : None


In [201]:
players_df['value'] = players_df['value'].apply(lambda x : convert_value_to_int(x))

## Cleaning age, number, country and position

In [202]:
print(players_df['country'].isna().value_counts())
print(players_df['age'].isna().value_counts())

country
False    18810
Name: count, dtype: int64
age
False    18810
Name: count, dtype: int64


In [221]:
# Identify rows where the 'number' column is not an integer
non_int_mask = pd.to_numeric(players_df['number'], errors='coerce').isna()

players_df_2= players_df.loc[non_int_mask, 'number']
players_df_2.value_counts()

number
-    616
Name: count, dtype: int64

In [229]:
players_df['number'] = players_df['number'].apply(lambda x : None if x == '-' else x)

In [232]:
# Identify rows where the 'age' column is not an integer
non_int_mask = pd.to_numeric(players_df['age'], errors='coerce').isna()

players_df_2= players_df.loc[non_int_mask, 'age']
players_df_2.value_counts()

age
-    45
Name: count, dtype: int64

In [233]:
players_df['age'] = players_df['age'].apply(lambda x : None if x == '-' else x)

In [238]:
print(players_df['country'].value_counts())


country
France           679
Brazil           640
Spain            633
Serbia           604
Ukraine          469
                ... 
Barbados           1
Liechtenstein      1
Cuba               1
Malawi             1
China              1
Name: count, Length: 159, dtype: int64


In [242]:
# Identify rows where the 'country' column is not a string
non_string_mask = ~players_df['country'].apply(lambda x: isinstance(x, str))

players_df_2= players_df.loc[non_string_mask, 'country']
players_df_2

Series([], Name: country, dtype: object)

In [244]:
# Identify rows where the 'position' column is not a string
non_string_mask = ~players_df['position'].apply(lambda x: isinstance(x, str))

players_df_2= players_df.loc[non_string_mask, 'position']
players_df_2

Series([], Name: position, dtype: object)

In [247]:
players_df.to_csv("tfmkt_scraper/crawled_data/players.csv", na_rep='NaN', index=False)

# Cleaning clubs dataframe

In [248]:
clubs_df = pd.read_json("tfmkt_scraper/crawled_data/clubs.json")
clubs_df[:5]

,league,name,href,value
0,CyprusLeague,Pafos FC,https://www.transfermarkt.com/pafos-fc/kader/v...,€24.15m
1,CyprusLeague,Aris Limassol,https://www.transfermarkt.com/aris-limassol/ka...,€20.00m
2,CyprusLeague,Omonia Nicosia,https://www.transfermarkt.com/omonia-nicosia/k...,€19.23m
3,CyprusLeague,APOEL Nicosia,https://www.transfermarkt.com/apoel-nicosia/ka...,€14.85m
4,CyprusLeague,AEK Larnaca,https://www.transfermarkt.com/aek-larnaca/kade...,€13.98m


In [252]:
for value in clubs_df['value'] :
    print(value)
    convert_value_to_int(value)

€24.15m
€20.00m
€19.23m
€14.85m
€13.98m
€12.43m
€11.25m
€9.43m
€6.50m
€5.85m
€5.53m
€4.58m
€3.80m
€2.88m
€11.30m
€9.70m
€8.60m
€5.70m
€4.90m
€4.05m
€3.80m
€3.80m
€3.55m
€3.40m
€3.20m
€2.75m
€23.18m
€22.91m
€14.23m
€9.71m
€7.48m
€7.39m
€7.18m
€7.05m
€5.93m
€5.89m
€5.78m
€5.38m
€5.05m
€4.40m
€28.78m
€13.20m
€11.75m
€8.68m
€6.43m
€5.55m
€5.33m
€4.73m
€4.18m
€3.68m
€3.55m
€3.05m
€14.40m
€10.70m
€9.80m
€5.75m
€5.01m
€4.90m
€4.60m
€3.50m
€3.30m
€2.71m
€11.48m
€10.88m
€6.15m
€5.90m
€3.80m
€2.54m
€2.36m
€2.15m
€2.13m
€1.98m
€20.23m
€12.03m
€11.75m
€11.58m
€10.68m
€5.60m
€5.53m
€5.48m
€4.20m
€3.03m
€4.42m
€4.01m
€4.01m
€3.88m
€3.75m
€3.49m
€3.25m
€3.01m
€2.78m
€2.77m
€2.58m
€2.25m
€5.75m
€4.75m
€4.63m
€4.45m
€4.35m
€4.30m
€3.98m
€3.88m
€3.55m
€3.53m
€5.30m
€5.15m
€4.83m
€4.61m
€3.94m
€3.10m
€3.00m
€2.98m
€2.45m
€2.04m
€6.19m
€5.14m
€4.75m
€4.31m
€4.02m
€3.79m
€2.86m
€2.18m
€1.97m
€1.15m
€4.81m
€4.62m
€3.66m
€3.30m
€2.66m
€2.54m
€2.48m
€2.35m
€1.82m
€1.34m
€4.11m
€3.51m
€3.37m
€2.50m
€2.04m
€2.0

UnboundLocalError: cannot access local variable 'value_int' where it is not associated with a value

In [249]:
clubs_df['value'] = players_df['value'].apply(lambda x : convert_value_to_int(x))

TypeError: 'float' object is not subscriptable